This time we employ the cross validation to figure out the best model for spam filter.

**Remark** The objective functions for logistic regression implemented in `sklearn` are:
<img src="L1.png">
and
<img src="L2.png">

where
- $w$ are the coefficients, which was denoted by $\beta_i$ in the class.
- $c$ is the intercept, which was denoted by $\beta_0$ in the class. We can change the parameter "fit_intercept" to keep or remove it.
- $C$ is the inverse of regularization strength. This is opposite to the $\alpha$ we used in Ridge and Lasso. Smaller values specify stronger regularization.
- Therefore the first objective function is of $L_1$ panelty and the second of $L_2$.

### Problem 1
Use the class <code>GridSearchCV</code> to find out the best combination of parameter for logistic regression. (Set <code>cv=5</code> and <code>scoring='accuracy'</code>). 

In [34]:
from __future__ import print_function
import pandas as pd
import numpy as np

spam_train_df = pd.read_csv('data/spam_train.csv')
x_train = spam_train_df.iloc[:, :57].values
y_train = spam_train_df.iloc[:, -1].values

spam_test_df = pd.read_csv('data/spam_test.csv')
x_test = spam_test_df.iloc[:, :57].values
y_test = spam_test_df.iloc[:, -1].values

In [35]:
from sklearn import model_selection
from sklearn import linear_model
para_grid = [{
    'penalty': ['l1', 'l2'],
    'fit_intercept': [False, True],
    'C': np.logspace(-5, 5, 100)
}]
logit = linear_model.LogisticRegression(random_state=42) # Use this random state for reproducible results

# Your solution

para_search = model_selection.GridSearchCV(logit, para_grid, cv=5, scoring='accuracy')
%time para_search.fit(x_train, y_train)

CPU times: user 45.7 s, sys: 505 ms, total: 46.2 s
Wall time: 47.2 s


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'penalty': ['l1', 'l2'], 'fit_intercept': [False, True], 'C': array([  1.00000e-05,   1.26186e-05, ...,   7.92483e+04,   1.00000e+05])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

    - What is the best combination?
    - What is the best score?
    - Refit the best estimator on the whole data set. How many coefficients were reduced to 0? (Hint: the absolute value of coefficients that are smaller than 1e-3.) 
    - What's the corresponding training error and test error? (Training error is the model performance on spam_train, while test error is the performance on spam_test.)

In [36]:
### your solution
# 1.
print(para_search.best_params_)

# 2.
print('Best score: {}'.format(para_search.best_score_))

# 3.
logit_best = para_search.best_estimator_
almost_zero = np.abs(logit_best.coef_) < 1e-3
print('Number of coefficients reduced to zero: {}'.format(np.sum(almost_zero)))
# 4.
print("Training error: %.5f" % (1 - logit_best.score(x_train, y_train)))
print("Training error: %.5f" % (1 - logit_best.score(x_test, y_test)))

# 5.
coef_accuracy = logit_best.coef_

{'C': 46.415888336127821, 'fit_intercept': True, 'penalty': 'l1'}
Best score: 0.928695652173913
Number of coefficients reduced to zero: 3
Training error: 0.06217
Training error: 0.07084


### Problem 2

Set *scoring = 'roc_auc'* and search again, what's the best parameters? Fit the best estimator on the spam_train data set. What's the training error and test error?

In [37]:
### your solution
y_train_bin = np.where(y_train == 'spam', 1, 0)
y_test_bin = np.where(y_test == 'spam', 1, 0)

para_search = model_selection.GridSearchCV(logit, para_grid, cv=5, scoring='roc_auc')

%time para_search.fit(x_train, y_train_bin)

CPU times: user 40.8 s, sys: 370 ms, total: 41.2 s
Wall time: 41.7 s


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'penalty': ['l1', 'l2'], 'fit_intercept': [False, True], 'C': array([  1.00000e-05,   1.26186e-05, ...,   7.92483e+04,   1.00000e+05])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [38]:
para_search.best_params_

{'C': 1.4174741629268048, 'fit_intercept': True, 'penalty': 'l1'}

In [39]:
# Note that para_search.score calls roc_auc on the best fit model. On the other hand,
# para_search.best_estimator_.score is hardcoded to return the accuracy
print("Training error: %.5f" % (1 - para_search.score(x_train, y_train_bin)))
print("Training error: %.5f" % (1 - para_search.score(x_test, y_test_bin)))

Training error: 0.02155
Training error: 0.03032


### Problem 3

In this exercise, we will predict the number of applications received(*Apps*) using the other variables in the College data set.

The features and the target variable are prepared as $x$ and $y$.

In [40]:
import pandas as pd
college = pd.read_csv('data/college.csv')
x = college.iloc[:, 2:]
y = college.iloc[:, 1]
print(college.shape)
college.head()

(777, 18)


,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


- (1) Split this data into a training set and a test set with train_size=0.5. (Hint: Use the function **sklearn.cross_validation.train_test_split** , set *random_state=0* and *train_size=0.5*.)

- (2) Fit a linear model on the training set and report the training error and test error. Report mean squared error only for this problem. (You can use the function *sklearn.metrics.mean_squared_error*).

- (3) Fit a ridge regression on the training set, with $\alpha$ chosen by the cross validation. Report the training error and test error.

- (4) Fit a lasso on the training set, with $\alpha$ chosen by the cross validation. Report the training error and test error

- (5) Compare the results obtained, what do you find?

In [41]:
### your solution
from sklearn import model_selection
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, train_size=0.5, random_state=0)
# (2)
import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error
ols = lm.LinearRegression().fit(x_train, y_train)
fmt = 'Training error of {0} model is: {1:.2f}     Test error of {0} model is: {2:.2f}'
train_linear_mse = mean_squared_error(ols.predict(x_train), y_train)
test_linear_mse = mean_squared_error(ols.predict(x_test), y_test)
print(fmt.format('linear', train_linear_mse, test_linear_mse))

Training error of linear model is: 1113145.99     Test error of linear model is: 1244800.27


/Users/rheineke/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [42]:
# (3)
param_grid = {'alpha': np.logspace(start=-5, stop=5, num=100)}

brute_ridge_cv = model_selection.GridSearchCV(lm.Ridge(), param_grid, cv=5)
%time brute_ridge_cv.fit(x_train, y_train)

# Some models can fit hyperparameters more efficiently than an exhaustive search
# http://scikit-learn.org/stable/modules/grid_search.html#model-specific-cross-validation
# We'll time these searches to see if there is any improvement for this small dataset

ridge_cv = lm.RidgeCV(alphas=param_grid['alpha'], cv=5)
%time ridge_cv.fit(x_train, y_train)

train_ridge_mse = mean_squared_error(ridge_cv.predict(x_train), y_train)
test_ridge_mse = mean_squared_error(ridge_cv.predict(x_test), y_test)
print('Alpha chosen: {:.2f}'.format(ridge_cv.alpha_))
print(fmt.format('ridge', train_ridge_mse, test_ridge_mse))

CPU times: user 2.6 s, sys: 38.7 ms, total: 2.63 s
Wall time: 2.56 s
CPU times: user 2.52 s, sys: 41.4 ms, total: 2.56 s
Wall time: 2.63 s
Alpha chosen: 756.46
Training error of ridge model is: 1113772.12     Test error of ridge model is: 1240199.69


In [43]:
# (4)
brute_lasso_cv = model_selection.GridSearchCV(lm.Lasso(), param_grid, cv=5)
%time brute_lasso_cv.fit(x_train, y_train)

lasso_cv = lm.LassoCV(alphas=param_grid['alpha'], cv=5)
%time lasso_cv.fit(x_train, y_train)
train_lasso_mse = mean_squared_error(lasso_cv.predict(x_train), y_train)
test_lasso_mse = mean_squared_error(lasso_cv.predict(x_test), y_test)
print('Alpha chosen: {:.2f}'.format(lasso_cv.alpha_))
print(fmt.format('lasso', train_lasso_mse, test_lasso_mse))

CPU times: user 2 s, sys: 22 ms, total: 2.02 s
Wall time: 2.04 s
CPU times: user 66.7 ms, sys: 1.65 ms, total: 68.3 ms
Wall time: 67.6 ms
Alpha chosen: 73.91
Training error of lasso model is: 1114287.76     Test error of lasso model is: 1235860.02


(5)
- Ridge and Lasso performs worse than linear model on the training set because of the regularization.
- But ridge and lasso predict better on the test set.
- Lasso performs worst on the training set but performs best on the test set.
- It's dangerous to use training error estimate test error.

### Problem 4
This time  we will try to predict the variable *Private* using the other variables in the College data set. The features and target variable are prepared for you.

In [44]:
x = college.iloc[:, 1:]
y = college.iloc[:, 0]

- (1) Split this data into a training set and a test set with train_size=0.5 (Hint: Use the function **sklearn.cross_validation.train_test_split**, set *random_state=1* and *train_size=0.5*.)]

- (2) Fit a logistic regression with regularizaton. Use the function **GridSearchCV** to fint out the best parameters.

In [45]:
grid_para_logit = [{'penality': ['l1', 'l2'], 'alpha': np.logspace(-5, 5, 100)}]

    - What's the best parameters?
    - Refit the model on the training set with best parameters. What's the training error and test error?
    
- (3) Fit a KNN model. Use the function **GridSearchCV** to determine the appropriate parameter *n_neighbors*. Refit the model on the training set and report the training error and test error.

- (4) Compare the results of logistic regression and KNN.

In [46]:
### your solution

## (1)
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, train_size=0.5, random_state=1)

## (2)
logit = lm.LogisticRegression()
grid_para_logit = [{
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-5, 5, 100)
}]
grid_search_logit = model_selection.GridSearchCV(logit, grid_para_logit, scoring='accuracy') ## search
%time grid_search_logit.fit(x_train, y_train)
logit = grid_search_logit.best_estimator_

print("The most appropriate parameter are: " + str(grid_search_logit.best_params_))
print("The training error of logistic: %.5f" %(1 - logit.score(x_train, y_train)))
print("The test     error of logistic: %.5f" %(1 - logit.score(x_test, y_test)))


/Users/rheineke/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


CPU times: user 8.01 s, sys: 51 ms, total: 8.06 s
Wall time: 8.14 s
The most appropriate parameter are: {'C': 0.027185882427329403, 'penalty': 'l1'}
The training error of logistic: 0.04381
The test     error of logistic: 0.05913


In [47]:
## (3)
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier()
grid_para_knn = [{'n_neighbors': range(3, 31)}]
gs_cv = model_selection.GridSearchCV(knn, grid_para_knn, scoring='accuracy')
grid_search_knn = gs_cv.fit(x_train, y_train)
print(grid_search_knn.best_params_)
knn = grid_search_knn.best_estimator_
print("The training error of KNN: %.5f" %(1 - knn.score(x_train, y_train)))
print("The test     error of KNN: %.5f" %(1 - knn.score(x_test, y_test)))

{'n_neighbors': 26}
The training error of KNN: 0.06186
The test     error of KNN: 0.07198


(4)
+ The test error is slightly higher than training error both in logistic regression and KNN.
+ Logistic regression is slightly better than KNN in this case.